In [3]:
pip install fosforml

You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install statsmodels

You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [6]:
from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()
 
table_name = '"MASTER_DATA"'
 
sf_df = my_session.sql("select * from {}".format(table_name))
df = sf_df.to_pandas()

In [7]:
df.columns

Index(['PRODUCT_CODE', 'PRODUCT_DESCRIPTION', 'CATEGORY_CODE', 'CATEGORY',
       'MANUFACTURING_LT', 'MFG_COST', 'LABOR_COST', 'PROFIT_MARGIN',
       'CUSTOMER_CODE', 'PRODUCT_ID', 'Production_Cost', 'Profit', 'Sales',
       'SUM_Part_Cost', 'CUSTOMER_NAME', 'CITY', 'STATE_PROVINCE',
       'COUNTRY_CODE', 'TRANSIT_LT', 'ORDER_ID',
       'Fact_Customer_Orders_PRODUCT_ID', 'ORDER_QTY', 'ORDER_DATE',
       'PART_CODE', 'QPA', 'UOM', 'UNIT_PRICE', 'LEAD_TIME_IN_WEEKS', 'MOQ',
       'PROD_LIFE_CYCLE'],
      dtype='object')

In [8]:
data = df[['ORDER_ID','Fact_Customer_Orders_PRODUCT_ID','ORDER_DATE','ORDER_QTY']]

In [9]:
# Convert ORDER_DATE to datetime if it's not already
data['ORDER_DATE'] = pd.to_datetime(data['ORDER_DATE'])

In [10]:
data_grouped = data.groupby(['Fact_Customer_Orders_PRODUCT_ID',data['ORDER_DATE'].dt.to_period('M')]).agg(Total_Order_Qty=('ORDER_QTY','sum')).reset_index()

In [11]:
data_grouped['ORDER_MONTH']=data_grouped['ORDER_DATE'].dt.to_timestamp()

In [12]:
data_grouped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2992 entries, 0 to 2991
Data columns (total 4 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   Fact_Customer_Orders_PRODUCT_ID  2992 non-null   object        
 1   ORDER_DATE                       2992 non-null   period[M]     
 2   Total_Order_Qty                  2992 non-null   int64         
 3   ORDER_MONTH                      2992 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(1), period[M](1)
memory usage: 93.6+ KB


In [13]:
data_grouped = data_grouped[['Fact_Customer_Orders_PRODUCT_ID','ORDER_MONTH','Total_Order_Qty']]

KeyError: "['PRODUCT_ID'] not in index"